In [1]:
import pandas as pd
topics = [
    'Business', 'Culture', 'Economy', 'Education', 'Energy', 
    'Engineering', 'Entertainment', 'Events', 'Food and drink', 'Geography', 
    'Government', 'Health', 'History', 'Human nature', 'Humanities', 'Industry', 
    'Language', 'Law', 'Life', 'Mass media', 'Mathematics', 'Military', 
    'Music', 'Nature', 'Organizations', 'People', 'Policy', 'Politics', 
    'Religion', 'Science and technology', 'Society', 'Sports', 'Universe', 'World'
]

In [2]:
%%time
import urllib.parse as ul
import trident
g = trident.Db('/scratch/kruit/kb/dbpedia_20201001/categories.tridentdb')

topic_subcats = {}
topic_articles = {}
for topic in topics:
    topic = topic.replace(" ","_")
    c = g.lookup_id(f'<http://dbpedia.org/resource/Category:{topic}>')
    if not c:
        continue
        
    br = g.lookup_id('<http://www.w3.org/2004/02/skos/core#broader>')
    subj = g.lookup_id('<http://purl.org/dc/terms/subject>')

    arts = set()
    cats = set([c])
    arts.update(g.s(subj, c))
    for c2 in g.s(br, c):
        arts.update(g.s(subj, c2))
        cats.add(c2)
        for c3 in g.s(br, c2):
            if c3 not in cats:
                arts.update(g.s(subj, c3))
                cats.add(c3)
                for c4 in g.s(br, c3):
                    if c4 not in cats:
                        arts.update(g.s(subj, c4))
    
    topic_subcats[topic] = cats
    topic_articles[topic] = arts
    print(f"{topic:>30s}: {len(cats):5d} subcats, {len(arts):8d} articles")

                      Business:   454 subcats,    73164 articles
                       Culture:   886 subcats,   404422 articles
                       Economy:   352 subcats,    54204 articles
                     Education:   523 subcats,    50648 articles
                        Energy:   441 subcats,    18630 articles
                   Engineering:   195 subcats,    38870 articles
                 Entertainment:   981 subcats,   132142 articles
                        Ethics:   245 subcats,    38774 articles
                        Events:   789 subcats,    51487 articles
                Food_and_drink:   647 subcats,    41434 articles
                     Geography:   246 subcats,    44074 articles
                    Government:  1064 subcats,   141264 articles
                        Health:   587 subcats,   112370 articles
                       History:   516 subcats,    85833 articles
                  Human_nature:   127 subcats,  1040093 articles
                    Human

In [3]:
%%capture --no-display
from dask.distributed import Client
client = Client(address = 'tcp://192.168.62.207:8786')
client

Client Scheduler: tcp://192.168.62.207:8786 Dashboard: http://192.168.62.207:8787/status,Cluster Workers: 10 Cores: 10 Memory: 673.47 GB


In [4]:
import dask.bag as db
from takco.util import robust_json_loads_lines
fnames = 'hdfs:///user/kruit/output/tabel-small-4/9-triples/*'
bag = db.read_text(fnames).map_partitions(robust_json_loads_lines).persist()
bag.count().compute()

4103

In [5]:
# Top header cells

from collections import Counter

def frequencies(bag, func):
    extract = bag.map_partitions(lambda ts: [x for t in ts for x in func(t)] )
    return Counter(dict(extract.frequencies().compute()))


def get_header_cells(t):
    for hr in t.get('tableHeaders'):
        for c in hr:
            text = c.get('text', '').strip()
            if text and text[0] != '_':
                yield text

def top_headers(bag):
    return pd.Series(dict(frequencies(bag, get_header_cells).most_common(10)))

top_headers(bag)

Year      620
Team      420
Date      415
Name      410
Title     348
W         333
L         328
Notes     283
Result    248
Player    230
dtype: int64

In [6]:
import trident
g = trident.Db('/export/scratch1/home/kruit/20200713-prop-skos')
plabel = g.lookup_id('<http://www.w3.org/2004/02/skos/core#prefLabel>')
def label(uri):
    i = g.lookup_id(f"<{uri}>")
    if i:
        for li in g.o(i, plabel):
            l = g.lookup_str(li).strip()
            if l.endswith('@en'):
                return l[1:-4]

In [7]:
# Top classes
from collections import Counter
bad = ['Q17442446','Q12139612','Q15633587','Q14204246','Q4167410','Q48522', 'Q11266439', 'Q4167836', 'Q13406463', 
       'Q21025364', 'Q56248902']

def get_coltypes(t):
    for _, cs in t.get('classes', {}).items():
        for c, score in cs.items():
            if not any(c.endswith(b) for b in bad):
                yield c

ent_name = lambda uri: uri.split('/')[-1] + " " + (label(uri) or '')
                
def top_coltypes(bag):
    freqs = frequencies(bag, get_coltypes).most_common(10)
    return pd.Series({ent_name(cls): count for cls, count in freqs})

top_coltypes(bag)

XMLSchema#string             10144
XMLSchema#decimal             6769
XMLSchema#dateTime            2303
Q13406554 sports festival      549
Q27020041 sports season        484
owl#Thing                      483
Q5 human                       478
Q215627 person                 478
Q164509 omnivore               478
Q154954 natural person         478
dtype: int64

In [8]:
wikiprops = '/export/scratch1/home/kruit/nary/data/kb/wikidata/wikidata-properties.txt'
puri_name = {
    l.split(' ', 1)[0]: l 
    for l in map(str.strip, open(wikiprops))
}

In [9]:
# Top classes
from collections import Counter

def get_props(t):
    for _, cps in t.get('properties', {}).items():
        for _, ps in cps.items():
            for p, score in ps.items():
                yield p.split('/')[-1]

def top_props(bag):
    freqs = frequencies(bag, get_props).most_common(10)
    return pd.Series({puri_name.get(prp, prp): count for prp, count in freqs})
                
top_props(bag)

P161 cast member                                         37
P54 member of sports team                                33
P131 located in the administrative territorial entity    27
P1346 winner                                             25
P1344 participant of                                     21
P1923 participating team                                 20
P175 performer                                           19
P301 category's main topic                               12
P57 director                                             11
P910 topic's main category                               11
dtype: int64

In [10]:
def get_pivots(prov):
    if 'pivot' in prov:
        yield prov['pivot']
    elif 'concat' in prov:
        for c in prov['concat']:
            yield from get_pivots(c)
            
def get_table_pivots(t):
    for pivot in get_pivots(t.provenance):
        yield from pivot['headers'][pivot['level']][pivot['colfrom']:pivot['colto']+1]

def top_pivots(bag):
    return pd.Series(dict(frequencies(bag, get_table_pivots).most_common(10)))
        
top_pivots(bag)

1        1568
3        1425
2        1415
4        1383
Total    1002
5         991
7         968
6         959
Final     946
8         939
dtype: int64

In [11]:
import pandas as pd



def get_pagetitles(prov):
    if 'pgTitle' in prov:
        yield prov['pgTitle']
    elif 'concat' in prov:
        for c in prov['concat']:
            yield from get_pagetitles(c)
        
def filter_tables(ts, articles):
    tables = []
    for t in ts:
        if articles & set(get_pagetitles(t.provenance)):
            tables.append(t)
    return tables
        
        
import trident
catdb = trident.Db('/scratch/kruit/kb/dbpedia_20201001/categories.tridentdb')
def get_article_titles(arts):
    for a in arts:
        yield catdb.lookup_str(a)[1:-1].replace('http://dbpedia.org/resource/', '').replace('_', ' ')

n_pages = bag.map(lambda t: list(get_pagetitles(t.provenance))).flatten().distinct().count().compute()
print(f"Got a total of {n_pages} pages")
        
ntopics = len(topic_articles)
for ti, (topic, articles) in enumerate(sorted(topic_articles.items(), key=lambda ta: len(ta[1]))):
    article_titles = set(get_article_titles(articles))
    print(topic, f"({len(article_titles)} articles)", f"[{ti+1}/{ntopics}]")
    tables = bag.map_partitions(filter_tables, article_titles).persist()
    ntables = tables.count().compute()
    print(ntables, 'tables')
    if ntables:
    
        
        headers = top_headers(tables)
        print('headers')
        display.display(headers)
        
        coltypes = top_coltypes(tables)
        print('coltypes')
        display.display(coltypes)
        
        props = top_props(tables)
        print('props')
        display.display(props)
        
        pivots = top_pivots(tables)
        print('pivots')
        display.display(pivots)
        
    print()

Got a total of 25699 pages
Universe (11963 articles)
11 tables
headers


TWh                    4
Facility               2
Preceded by            1
Rank                   1
Country/Region         1
Date of information    1
Capita                 1
Prim. energy           1
Production             1
Export                 1
dtype: int64

coltypes


XMLSchema#string                         25
XMLSchema#decimal                        17
owl#Thing                                 3
XMLSchema#dateTime                        3
Q215627 person                            1
Q5 human                                  1
Q164509 omnivore                          1
Q154954 natural person                    1
Q21025364 WikiProject                     1
Q56248902 Wikimedia community project     1
dtype: int64

props


...

XMLSchema#string                     1617
XMLSchema#decimal                     741
XMLSchema#dateTime                    540
Q5 human                              414
Q215627 person                        414
Q164509 omnivore                      414
Q154954 natural person                414
Q7725310 series of creative works      99
Q1631107 bibliography                  97
Q273057 discography                    95
dtype: int64

props


P161 cast member             25
P54 member of sports team    19
P175 performer               17
P735 given name               6
P40 child                     5
P361 part of                  5
P22 father                    4
P39 position held             4
P57 director                  3
P197 adjacent station         3
dtype: int64

pivots


Total               887
1                   611
3                   604
2                   589
4                   587
National Rail       508
League              490
US                  437
Disused railways    394
Line                338
dtype: int64